In [1]:
import pandas as pd
import nasdaqdatalink
nasdaqdatalink.ApiConfig.api_key ='ABH2M7eCbJJXy7YS_Cj7'

In [2]:
var = pd.read_csv('Variables.csv')
var.set_index('Date', inplace=True)
ticker = pd.read_csv('ticker.csv')
tick_list = ticker.columns.tolist()
start = "2010-01-01"
end = "2017-01-01"

In [7]:
df1 = nasdaqdatalink.get_table('WIKI/PRICES', date = { 'gte': start, 'lte': end },
                              paginate=True,qopts={"columns":["adj_close","date","ticker"]},
                              ticker=tick_list[:251])

df2 = nasdaqdatalink.get_table('WIKI/PRICES', date = { 'gte': start, 'lte': end },
                              paginate=True,qopts={"columns":["adj_close","date","ticker"]},
                              ticker=tick_list[251:])
# df upside down
df1 = df1.iloc[::-1]
df1.reset_index(drop=True, inplace=True)
df2 = df2.iloc[::-1]
df2.reset_index(drop=True, inplace=True)

df = pd.concat([df1,df2])
# pivot table is like spreadsheet
# specify adj_close as values to avoid multi index
df = df.pivot_table(index='date',columns='ticker',values='adj_close')
df = df.pct_change()

In [ ]:
# extract/apply the smallest timeframe in both files before merging
var.index = pd.to_datetime(var.index)
var = var[var.index.isin(df.index)]
df = df[df.index.isin(var.index)]

In [26]:
mega = pd.concat([df, var], axis=1)
# some ticker from wiki doesn't have sufficient data price from nasdaq -> drop
mega = mega.dropna(axis=1)
mega.to_csv("StockData.csv", encoding="UTF-8")